## <center>Classic Plotly animation vs animation with ipywidgets </center>##

We reproduce here the animation illustrated in a gif file shared bt Fermat's Library (@fermatslibrary) on twitter
 [https://twitter.com/fermatslibrary/status/1040951633692319744](https://twitter.com/fermatslibrary/status/1040951633692319744).
 
Let us describe what is observed in that gif: An inner circle of radius r, with 8 fixed points is rolling inside a base circle of radius R=2r.

Let us fix $R=1$. Then the white rays intersect the base circle  at the points representing the $16^{th}$ roots of the unity. 
Let us say that at the time t=0, the rolling circle, containing eight marked dots, has the center at $(0.5, 0)$. Hence the dots can be given as points in the  complex plane:    $u[k]=0.5+0.5\cdot e^{2\pi j k/8}$, $k=\overline{0,7}$ (where $e^{2\pi j k/8}$ are the $8^{th}$ roots of  unity). 

The $m^{th}$ frame of the animation displays  the position of  the dots obtained from the initial ones, as follows:
  - rotate the center c=0.5+0j about O, by $\theta_m=2 m \pi/16$ and get the new center $c_m$;
  - place  on the circle of center $c_m$ and radius r=1/2, the points corresponding to $u[k]-r$,  rotated about O with $-\theta_m$, i.e.  plot the dots $w[k]=e^{m\pi j/8}\dot c+e^{-m\pi j/8}(u[k]-r)$, $k=\overline{0,7}$

In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode,  iplot, plot
init_notebook_mode(connected=True)

In [2]:
import numpy as np
from numpy import pi
import plotly.graph_objs as go
import ipywidgets as ipw

In [3]:
c=0.5+1j*0
r=0.5

In [4]:
u = np.array([c+r*np.exp(2*k*np.pi*1j/8) for k in range(8)], dtype=np.complex)

In [5]:
data = [dict(type='scatter',
             x=u.real,
             y=u.imag,
             mode='markers',
             marker=dict(size=15, color='white'),
             name='moving_pts')]

In [6]:
a = 1.01
axis = dict(visible=False,
            range=[-a,a],
            autorange=False,
            title='')

layout = dict(title="Fermat's Library Animation",
              autosize=False,
              width=500,
              height=445,
              showlegend=False,
              xaxis=dict(axis),
              yaxis=dict(axis),
              hovermode='closest',
              margin=dict(b=5),
              shapes=[],
        
              updatemenus=[dict(type='buttons',
                                showactive=False,
                                y=1,
                                x=1.25,
                                xanchor='right',
                                yanchor='top',
                                pad=dict(l=10),
                                buttons=[dict(label='Play',
                                              method='animate',
                                              args=[None, dict(frame=dict(duration=200, redraw=False), 
                                                    transition=dict(duration=100),
                                                    fromcurrent=True,
                                                    mode='immediate')] )] )])

In [7]:
z=np.array([np.exp(2*k*np.pi*1j/16) for k in range(16)], dtype=np.complex) # the cmplx points representing the $!6^{th}$ 
                                                                           # roots of unity

In [8]:
R = 1.005
layout['shapes'].append(dict(type='circle',
                             layer='below',
                             xref='x',
                             yref='y',
                             fillcolor= 'rgba(10,10,10, 0.9)',
                             x0= -R,
                             y0= -R,
                             x1= R,
                             y1= R,
                             line= dict(color= 'white')))

#Define the 8 diameters in the circle of center 0+1j*0 as quadratic Bezier curves with colinear control points
for k in range( 8):
    x0 = z[k].real
    y0 = z[k].imag
    x1 = z[k+8].real
    y1 = z[k+8].imag
    layout['shapes'].append(dict(type='path',
                                 layer='below',
                                 path= f"M{x0}, {y0} Q 0, 0 {x1}, {y1}",                    
                                 line= dict(color= 'white', width=0.5)))   

In [10]:
frames = []
for m in range(65):
    # rotate the initial center c=5+1j*0 with -m*2 pi/16 and get the new center C_m; 
    # the points (u[k]-0.5) are placed on the circle (C_m, r=0.5) rotated with m*pi/8
    w = np.exp(-1j*m*pi/8)*c+np.exp(1j*m*pi/8)*(u-r)#interchanging the signs of rotations we get 
                                                      #a clockwise rolling small circle
    frames.append(dict(data=[dict(type='scatter',
                                  x=w.real,
                                  y=w.imag)],
                       traces=[0]))

In [11]:
fig = go.Figure(data=data, layout=layout, frames=frames)  

Now let us generate the same animation using ipywidgets:

In [12]:
fw = go.FigureWidget(data=data, layout=layout)

In [13]:
fw.layout.updatemenus=None

In [14]:
slider = ipw.IntSlider(value=0, min=0, max=64, step=1, description='')
slider.layout = dict(margin='10px 80px 40px 5px', width='400px')

In [15]:
def points_changed(change):
    wn = np.exp(1j*slider.value*pi/8) * c + np.exp(-1j*slider.value*pi/8) * (u - r)
    with fw.batch_update():
        fw.data[0].x = list(wn.real)
        fw.data[0].y = list(wn.imag)
       
        
slider.observe(points_changed, 'value')

In [16]:
play_widget = ipw.Play(value=0, min=0, max=65, step=1, interval=200)
play_widget.layout = dict(margin='10px 10px 50px 100px')
ipw.jslink((play_widget, 'value'), (slider, 'value'))

Plotly "classical" animation:

In [17]:
iplot(fig)

In [18]:
ipw.VBox([fw, ipw.HBox([play_widget, slider])]) #Plotly animation via ipywidgets:

    'data': [{'marker': {'color': 'white', 'size': 15},
              'mode': 'm…

 The first animation illustrates the right motion of the eight points, while the second one keeps fixed the central dot and we cannot see a point comming to that position, in each step . 
 So the motion of each point along its own  diameter isn't noticeable here.
 
 What in ipywidgets animation leads to this behaviour?